In [1]:
%matplotlib widget

import numpy as np
import librosa
import librosa.display
from IPython.display import Audio
import scipy.io.wavfile as wavfile

from audio_lab.jupyter_utils import *
from audio_lab.daw import Daw
from audio_lab.measurement import *

# Set up the root logger and display its output in this cell.
logger = get_cell_logger()
logger.show_logs()

Button(description='Clear logs', style=ButtonStyle())

Output(layout=Layout(border='1px solid white', height='300px', overflow='auto'))

In [2]:
# Use a consistent sampling rate and fft size for generating the test signal and analyzing the response signal.
# This helps avoid aliasing issues.
sr=44100
n_fft=1024*32

In [6]:
# Generate a pickup test signal for use in other programs.
test_freqs = align_freqs(sr, n_fft, freqs_logspace(n_freqs=100))
test_signal = generate_1_f_signal(test_freqs, sr=sr, duration=15)
fig, ax = start_figure('Pickup test signal', figsize=(8,5))
plot_frequency_response(test_signal, sr, n_fft=n_fft, ax=ax, ylim=(-60, 2))
wavfile.write('pickup-testing/pickup-test-signal.wav', sr, test_signal)

# The signal peak is normalize to 1 (0 dB).
display(Audio(test_signal, rate=sr))

FigureCanvasNbAgg()

In [7]:
# Plot the frequency response of an output signal, as recorded in e.g. Audacity. You can export selected audio, 
# which is useful when you can process offline. It is also easier to estimate and compensate for audio delays.

signal, _ = librosa.load('pickup-testing/strat-bridge-series-wiring.wav', sr=sr, duration=3)
fig, ax = start_figure('Strat bridge series wiring', figsize=(8,5))
plot_frequency_response(signal, sr, n_fft=n_fft, ax=ax)

# You can hear the high end  
display(Audio(signal, rate=sr))

FigureCanvasNbAgg()

In [ ]:
# Measure live pickup frequency response using the second I/O channels of an audio interface
# and an inductor.

try:
    soundcard.terminate()
except NameError:
    pass

soundcard = measure_pickup_frequency_response(test_signal_channel_index=1,
                                              response_signal_channel_index=1,
                                              num_channels=2,
                                              n_fft=1024 * 32,
                                              sr=sr,
                                              signal_duration=3)

In [ ]:
# Estimate the transfer function between the input and output signals.
try:
    soundcard.terminate()
except NameError:
    pass

soundcard = plot_transfer_function(1, 0, num_channels=2, n_fft=4096*8, update_interval=2)

In [ ]:
# Play back audio, apply transforms, and display a live spectrogram.

try:
    daw.terminate()
except NameError:
    pass

fig, ax = start_figure("DAW Monitor", figsize=(8, 5))
daw = Daw()